In [1]:
import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

In [3]:
cv_only = True
save_cv = True
full_train = False

In [4]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True

In [5]:
path = '../input/porto-seguros-safe-driver-prediction-dataset/'

train = pd.read_csv(path+'train.csv')
train_label = train['target']
train_id = train['id']
test = pd.read_csv(path + 'test.csv')
test_id = test['id']

In [6]:
NFOLDS = 5
kfold = StratifiedKFold(n_splits = NFOLDS, shuffle = True, random_state = 218)

In [7]:
y = train['target'].values
drop_features = ['id', 'target']

X = train.drop(drop_features, axis = 1)
feature_names = X.columns.tolist()

In [8]:
cat_features = [c for c in feature_names if('cat' in c and 'count' not in c)]
num_features = [c for c in feature_names if ('cat' not in c and 'calc' not in c)]

In [9]:
train['missing'] = (train == -1).sum(axis=1).astype(float)
test['missing'] = (test == -1).sum(axis = 1).astype(float)
num_features.append('missing')

In [10]:
for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])
    
enc = OneHotEncoder()
enc.fit(train[cat_features])

OneHotEncoder()

In [11]:
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

In [12]:
ind_features = [c for c in feature_names if 'ind' in c]

In [13]:
count = 0
for c in ind_features:
    if count ==0:
        train['new_ind'] = train[c].astype(str) + '_'
        test['new_ind'] = test[c].astype(str)+'_'
        count += 1
    else:
        train['new_ind'] += train[c].astype(str) +'_'
        test['new_ind'] += test[c].astype(str) + '_'

In [14]:
cat_count_features = []

for c in cat_features + ['new_ind']:
    d = pd.concat([train[c], test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

In [15]:
train_list = [train[num_features + cat_count_features].values, X_cat,]
test_list = [test[num_features + cat_count_features].values, X_t_cat,]

In [16]:
X = ssp.hstack(train_list).tocsr()
X_test = ssp.hstack(test_list).tocsr()

In [17]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
           "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }

In [18]:
x_score = []
final_cv_train = np.zeros(len(train_label))
final_cv_pred = np.zeros(len(test_id))

In [19]:
for s in np.arange(16):
    break

In [20]:
cv_train = np.zeros(len(train_label))
cv_pred = np.zeros(len(test_id))

In [21]:
params['seed'] = s

In [22]:
kf = kfold.split(X, train_label)

In [23]:
best_trees = []
fold_scores = []

In [24]:
for i, (train_fold, validate) in enumerate(kf):
    break

In [25]:
X_train, X_validate, label_train, label_validate = \
X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
dtrain = lgbm.Dataset(X_train, label_train)
dvalid = lgbm.Dataset(X_validate, label_validate, reference = dtrain)
bst = lgbm.train(params, dtrain, num_boost_round, valid_sets = dvalid, feval = evalerror, verbose_eval = 100, early_stopping_rounds = 100)
best_trees.append(bst.best_iteration)
cv_pred += bst.predict(X_test, num_iteration = bst.best_iteration)
cv_train[validate] += bst.predit(X_validate)

score = Gini(label_validate, cv_Train[validate])
print score

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(score)? (<ipython-input-25-8c1d781bf0b7>, line 11)